# TFG - Anàlisi de l’abandonament escolar prematur a Catalunya
## Tractament de Dades Any 2018
Jahziel Hidalgo Artigas



## Taula de continguts<a class="anchor" id="index"></a>
* [Imports](#imports)
* [Funcions](#funcions)
* [Càrrega Datasets](#datasets)
* [Comarques](#comarques)
* [Dades Formació](#formacio)
    * [Anàlisi per Categoría](#formacio)
    * [Formació d'Adults](#formacio_adults)
* [Dades Població](#poblacio)
   * [Població per Gènere](#poblacio_genere)
   * [Població per Edat](#poblacio_edat)
   * [Atur per Edat](#atur)
   * [Atur per Gènere](#atur_genere)
   * [Percentatge/Ratio Estudiants](#pct_estudiants)
   * [Població per Nacionalitat](#poblacio_nacionalitat)
* [Dades Socioeconòmiques](#dades_socioeconomiques)
    * [Repetidors](#repetidors)
    * [Total Repetidors per Comarca](#repetidors_comarca)
    * [Impacte de la Nacionalitat](#extrangers)
    * [Nivell Socioeconòmic (Renda Familiar)](#renta_familiar)
    * [Nivell Socioeconòmic (RFBD](#rfbd)


### Imports<a class="anchor" id="imports"></a>

In [284]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
import os
from unidecode import unidecode

### Funcions<a class="anchor" id="funcions"></a>

In [258]:
def clean_dataframe(df):
    # Netejar les columnes: treure els accents, substituir els espais per guions baixos i convertir a minúscules
    df.columns = [unidecode(col.replace(' ', '_').lower()) for col in df.columns]
    
    # Netejar les dades: treure els accents, convertir a minúscules i processar només les cadenes de text
    df = df.apply(lambda x: x.apply(lambda val: unidecode(str(val)).lower() if isinstance(val, str) else val) 
                  if x.dtype == 'object' else x, axis=0)
    
    return df

def filtrar_comarques(df, columna):
    comarques_valides = [
        'alt camp', 'alt emporda', 'alt penedes', 'alt urgell', 'alta ribagorca', 
        'anoia', 'aran', 'bages', 'baix camp', 'baix ebre', 'baix emporda', 
        'baix llobregat', 'baix penedes', 'barcelones', 'bergueda', 'cerdanya', 
        'conca de barbera', 'garraf', 'garrigues', 'garrotxa', 'girones', 'maresme', 
        'moianes', 'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira', 
        "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre", 'ripolles', 
        'segarra', 'segria', 'selva', 'solsones', 'tarragones', 'terra alta', 'urgell', 
        'valles occidental', 'valles oriental'
    ]
    
    # Filtrar les files que continguin valors en la columna dins de la llista de comarques vàlides
    df = df[df[columna].isin(comarques_valides)].reset_index(drop=True)
    
    return df


### Càrrega Datasets<a class="anchor" id="datasets"></a>

In [260]:
df_comarques = pd.read_csv('csv/2018/comarques.csv', delimiter=';', skiprows=3) # Filtar per Nivell = 'Comarca'
df_formacio = pd.read_csv('csv/2018/nivell_formacio_2018.csv', delimiter=';', skiprows=5) 
df_formacio_adults = pd.read_csv('csv/2018/formacio_adults_2018.csv', delimiter=';', skiprows=5)
df_poblacio_genere = pd.read_csv('csv/2018/poblacio_genere_2018.csv', delimiter=';')
df_poblacio_edat = pd.read_csv('csv/2018/poblacio_edat_2018.csv', delimiter=';')
df_poblacio_nacionalitat = pd.read_csv('csv/2018/poblacio_nacionalitat_2018.csv', delimiter=';', skiprows=6)
df_repetidors = pd.read_csv('csv/2018/alumnes_repetidors_2018.csv', delimiter=';')
df_renta_familiar = pd.read_csv('csv/2018/renta_familiar2018.csv', delimiter=';', skiprows=6)
df_rfbd = pd.read_csv('csv/2018/RFBD_2018.csv', delimiter=';', skiprows=6)
df_atur_edat = pd.read_csv('csv/2018/atur_edat_2018.csv', delimiter=';', skiprows=6)
df_atur_genere = pd.read_csv('csv/2018/atur_genere_2018.csv', delimiter=';', skiprows=6)

In [8]:
datasets = {
    'Comarques': df_comarques,
    'Formació': df_formacio,
    'Formació Adults': df_formacio_adults,
    'Població Sexe': df_poblacio_genere,
    'Població Edat': df_poblacio_edat,
    'Població Nacionalitat': df_poblacio_nacionalitat,
    'Repetidors': df_repetidors,
    'Renta Familiar': df_renta_familiar,
    'RFBD': df_rfbd
}

# Iterar sobre el diccionari per mostrar les primeres files de cada dataset
for name, df in datasets.items():
    print(f"Primeres files de {name}:")
    display(df.head()) 
    print("\n") 

Primeres files de Comarques:


,Nivell,Codi,Nom
0,Comarca,1,Alt Camp
1,Municipi,430017,Aiguamúrcia
2,Municipi,430056,Alcover
3,Municipi,430108,Alió
4,Municipi,430347,Bràfim




Primeres files de Formació:


,comarca,Analfabets,Educació primària incompleta,Educació primària,Primera etapa d'educació secundària i similar,Segona etapa d'educació secundària amb orientació general,Segona etapa d'educació secundària amb orientació professional,Ensenyament de grau superior de formació professional i similars,Graus universitaris de fins a 240 crèdits i similars,Graus universitaris de més de 240 crèdits i similars,Màster universitari i similars,Doctorat universitari,Total
0,Alt Camp,120,1540,4530,13375,3950,4115,3785,2830,2315,565,185,37310
1,Alt Empordà,385,6630,15960,41995,17905,10575,6590,8670,7345,1505,575,118130
2,Alt Penedès,220,2365,12485,27875,10445,11075,8355,7875,6670,1430,540,89340
3,Alt Urgell,50,805,2120,5415,2640,1765,1415,1545,1295,360,110,17520
4,Alta Ribagorça,10,155,405,925,480,310,275,370,330,65,25,3340




Primeres files de Formació Adults:


...

,Ârea territorial codi,Àrea territorial,Comarca codi,Comarca,Municipi codi,Municipi,Districte codi,Naturalesa,Titularitat,Estudis,Nivell,Sexe,Concert,Resultat de l'avaluació,N Avaluats estudi
0,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,SI,Alumnes avaluats,3
1,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,SI,No promocionen,1
2,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,SI,Promocionen al mes de juny,1
3,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,SI,Promocionen amb pendents,1
4,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,HOME,SI,Alumnes avaluats,9




Primeres files de Renta Familiar:


,Unnamed: 0,RFDB (miles de euros),RFDB por habitante (euros),RFDB por habitante (Índex Catalunya=100 por habitante)
0,Alt Camp,688231,15537,"88,9"
1,Alt Empordà,1978509,14393,"82,3"
2,Alt Penedès,1765253,16340,"93,5"
3,Alt Urgell,276699,13713,"78,4"
4,Alta Ribagorça,57443,15051,"86,1"




Primeres files de RFBD:


,Unnamed: 0,Recursos. Remuneración asalariados,Recursos. Excedente bruto explotación,Recursos. Prestaciones sociales,Usos. Cotizaciones sociales,Usos. Impuestos
0,Alt Camp,544469,186056,191794,161595,89680
1,Alt Empordà,1449595,674267,450761,448546,295196
2,Alt Penedès,1449403,492449,471889,421447,255331
3,Alt Urgell,177640,97022,80091,56810,32616
4,Alta Ribagorça,38552,24013,14531,12735,8260


### Comarques<a class="anchor" id="comarques"></a>

L'estudi es farà per comarques, així que necessitem obtenir el codi de les comarques, el qual ens servirà per agrupar el conjunt final.

In [11]:
# Primerament, netegem el dataset per assegurar-nos que no hi hagi problemes amb els noms de les columnes, accents, etc.
df_comarques = clean_dataframe(df_comarques)

In [12]:
df_comarques.head()

,nivell,codi,nom
0,comarca,1,alt camp
1,municipi,430017,aiguamurcia
2,municipi,430056,alcover
3,municipi,430108,alio
4,municipi,430347,brafim


In [13]:
df_comarques.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nivell  990 non-null    object
 1   codi    990 non-null    int64 
 2   nom     990 non-null    object
dtypes: int64(1), object(2)
memory usage: 23.3+ KB


In [14]:
# Filtrarem les dades per seleccionar només aquelles files on el 'nivell' sigui 'comarca',
# i només seleccionarem les columnes 'codi' i 'nom' que seran necessàries per a l'anàlisi posterior.
df_comarques = df_comarques[df_comarques['nivell'] == 'comarca'][['codi', 'nom']].reset_index(drop=True)
# Convertim la columna 'codi' a format de dos dígits
df_comarques['codi'] = df_comarques['codi'].astype(str).str.zfill(2)
df_comarques.head()

,codi,nom
0,01,alt camp
1,02,alt emporda
2,03,alt penedes
3,04,alt urgell
4,05,alta ribagorca


In [15]:
# Afegim manualment el codi d'Aran (39)
df_comarques.loc[len(df_comarques)] = ['39', 'aran']
df_comarques = df_comarques.sort_values(by='codi').reset_index(drop=True)
df_comarques.tail()

,codi,nom
38,39,aran
39,40,valles occidental
40,41,valles oriental
41,42,moianes
42,43,llucanes


In [16]:
df_comarques['codi'] = df_comarques['codi'].astype(str)
df_comarques.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   codi    43 non-null     object
 1   nom     43 non-null     object
dtypes: object(2)
memory usage: 820.0+ bytes


In [17]:
# Comprovem els noms únics de les comarques per assegurar-nos que no hi ha comarques duplicades
df_comarques['nom'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'bages', 'baix camp', 'baix ebre',
       'baix emporda', 'baix llobregat', 'baix penedes', 'barcelones',
       'bergueda', 'cerdanya', 'conca de barbera', 'garraf', 'garrigues',
       'garrotxa', 'girones', 'maresme', 'montsia', 'noguera', 'osona',
       'pallars jussa', 'pallars sobira', "pla d'urgell",
       "pla de l'estany", 'priorat', "ribera d'ebre", 'ripolles',
       'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'aran', 'valles occidental',
       'valles oriental', 'moianes', 'llucanes'], dtype=object)

In [18]:
# Imprimim el nombre de comarques úniques que tenim en el dataset
print(f"Hi han {len(df_comarques['nom'].unique())} comarques")

Hi han 43 comarques


In [19]:
# Crear el diccionari de codis (comarca -> codi)
comarques_dict = df_comarques.set_index('nom')['codi'].to_dict()
comarques_dict

{'alt camp': '01',
 'alt emporda': '02',
 'alt penedes': '03',
 'alt urgell': '04',
 'alta ribagorca': '05',
 'anoia': '06',
 'bages': '07',
 'baix camp': '08',
 'baix ebre': '09',
 'baix emporda': '10',
 'baix llobregat': '11',
 'baix penedes': '12',
 'barcelones': '13',
 'bergueda': '14',
 'cerdanya': '15',
 'conca de barbera': '16',
 'garraf': '17',
 'garrigues': '18',
 'garrotxa': '19',
 'girones': '20',
 'maresme': '21',
 'montsia': '22',
 'noguera': '23',
 'osona': '24',
 'pallars jussa': '25',
 'pallars sobira': '26',
 "pla d'urgell": '27',
 "pla de l'estany": '28',
 'priorat': '29',
 "ribera d'ebre": '30',
 'ripolles': '31',
 'segarra': '32',
 'segria': '33',
 'selva': '34',
 'solsones': '35',
 'tarragones': '36',
 'terra alta': '37',
 'urgell': '38',
 'aran': '39',
 'valles occidental': '40',
 'valles oriental': '41',
 'moianes': '42',
 'llucanes': '43'}

### Dades Formació<a class="anchor" id="formacio"></a>

#### Anàlisi per Categoría<a class="anchor" id="formacio"></a>

In [262]:
df_formacio = pd.read_csv('csv/2018/nivell_formacio_2018.csv', delimiter=';', skiprows=5) 

In [264]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_formacio_clean = clean_dataframe(df_formacio)
df_formacio_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 13 columns):
 #   Column                                                            Non-Null Count  Dtype 
---  ------                                                            --------------  ----- 
 0   comarca                                                           55 non-null     object
 1   analfabets                                                        55 non-null     int64 
 2   educacio_primaria_incompleta                                      55 non-null     int64 
 3   educacio_primaria                                                 55 non-null     int64 
 4   primera_etapa_d'educacio_secundaria_i_similar                     55 non-null     int64 
 5   segona_etapa_d'educacio_secundaria_amb_orientacio_general         55 non-null     int64 
 6   segona_etapa_d'educacio_secundaria_amb_orientacio_professional    55 non-null     int64 
 7   ensenyament_de_grau_superior_de_formacio_profe

In [266]:
# Filtrar per comarques
df_formacio_clean = filtrar_comarques(df_formacio_clean, 'comarca')

In [268]:
# Comprovem els canvis realitzats
df_formacio_clean.info()
display(df_formacio_clean.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 13 columns):
 #   Column                                                            Non-Null Count  Dtype 
---  ------                                                            --------------  ----- 
 0   comarca                                                           42 non-null     object
 1   analfabets                                                        42 non-null     int64 
 2   educacio_primaria_incompleta                                      42 non-null     int64 
 3   educacio_primaria                                                 42 non-null     int64 
 4   primera_etapa_d'educacio_secundaria_i_similar                     42 non-null     int64 
 5   segona_etapa_d'educacio_secundaria_amb_orientacio_general         42 non-null     int64 
 6   segona_etapa_d'educacio_secundaria_amb_orientacio_professional    42 non-null     int64 
 7   ensenyament_de_grau_superior_de_formacio_profe

,comarca,analfabets,educacio_primaria_incompleta,educacio_primaria,primera_etapa_d'educacio_secundaria_i_similar,segona_etapa_d'educacio_secundaria_amb_orientacio_general,segona_etapa_d'educacio_secundaria_amb_orientacio_professional,ensenyament_de_grau_superior_de_formacio_professional_i_similars,graus_universitaris_de_fins_a_240_credits_i_similars,graus_universitaris_de_mes_de_240_credits_i_similars,master_universitari_i_similars,doctorat_universitari,total
0,alt camp,120,1540,4530,13375,3950,4115,3785,2830,2315,565,185,37310
1,alt emporda,385,6630,15960,41995,17905,10575,6590,8670,7345,1505,575,118130
2,alt penedes,220,2365,12485,27875,10445,11075,8355,7875,6670,1430,540,89340
3,alt urgell,50,805,2120,5415,2640,1765,1415,1545,1295,360,110,17520
4,alta ribagorca,10,155,405,925,480,310,275,370,330,65,25,3340
5,anoia,370,3500,12360,34370,11455,11880,9535,7655,6090,1360,410,98975
6,aran,15,150,710,2520,1530,1005,840,925,725,140,35,8595
7,bages,400,4875,20380,49380,17185,16090,12915,13225,10835,2410,780,148475
8,baix camp,1280,6765,17930,51385,20520,16490,13895,12920,11755,3125,1000,157070
9,baix ebre,185,3460,9885,23505,8185,6960,4860,4645,3735,1200,260,66885


In [270]:
# Comprovem els canvis realitzats
df_formacio_clean.info()
display(df_formacio_clean.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 13 columns):
 #   Column                                                            Non-Null Count  Dtype 
---  ------                                                            --------------  ----- 
 0   comarca                                                           42 non-null     object
 1   analfabets                                                        42 non-null     int64 
 2   educacio_primaria_incompleta                                      42 non-null     int64 
 3   educacio_primaria                                                 42 non-null     int64 
 4   primera_etapa_d'educacio_secundaria_i_similar                     42 non-null     int64 
 5   segona_etapa_d'educacio_secundaria_amb_orientacio_general         42 non-null     int64 
 6   segona_etapa_d'educacio_secundaria_amb_orientacio_professional    42 non-null     int64 
 7   ensenyament_de_grau_superior_de_formacio_profe

,comarca,analfabets,educacio_primaria_incompleta,educacio_primaria,primera_etapa_d'educacio_secundaria_i_similar,segona_etapa_d'educacio_secundaria_amb_orientacio_general,segona_etapa_d'educacio_secundaria_amb_orientacio_professional,ensenyament_de_grau_superior_de_formacio_professional_i_similars,graus_universitaris_de_fins_a_240_credits_i_similars,graus_universitaris_de_mes_de_240_credits_i_similars,master_universitari_i_similars,doctorat_universitari,total
0,alt camp,120,1540,4530,13375,3950,4115,3785,2830,2315,565,185,37310
1,alt emporda,385,6630,15960,41995,17905,10575,6590,8670,7345,1505,575,118130
2,alt penedes,220,2365,12485,27875,10445,11075,8355,7875,6670,1430,540,89340
3,alt urgell,50,805,2120,5415,2640,1765,1415,1545,1295,360,110,17520
4,alta ribagorca,10,155,405,925,480,310,275,370,330,65,25,3340
5,anoia,370,3500,12360,34370,11455,11880,9535,7655,6090,1360,410,98975
6,aran,15,150,710,2520,1530,1005,840,925,725,140,35,8595
7,bages,400,4875,20380,49380,17185,16090,12915,13225,10835,2410,780,148475
8,baix camp,1280,6765,17930,51385,20520,16490,13895,12920,11755,3125,1000,157070
9,baix ebre,185,3460,9885,23505,8185,6960,4860,4645,3735,1200,260,66885


In [272]:
# Definim el mapeig de les categories educatives
cat = {
    'analfabetisme_total': ['analfabets'],
    'educacio primaria incompleta_total': ['educacio_primaria_incompleta'],
    'educacio primaria_total': ['educacio_primaria'],
    'educacio secundaria_total': ['primera_etapa_d\'educacio_secundaria_i_similar', 
                            'segona_etapa_d\'educacio_secundaria_amb_orientacio_general',
                            'segona_etapa_d\'educacio_secundaria_amb_orientacio_professional'],
    'formacio professional_total': ['ensenyament_de_grau_superior_de_formacio_professional_i_similars'],
    'graus universitaris_total': ['graus_universitaris_de_fins_a_240_credits_i_similars', 
                            'graus_universitaris_de_mes_de_240_credits_i_similars'],
    'masters universitaris_total': ['master_universitari_i_similars'],
    'doctorat universitari_total': ['doctorat_universitari']
}

# Creem un nou DataFrame per emmagatzemar els resultats
df_grouped = pd.DataFrame()

# Iterem sobre les categories i sumem les columnes corresponents
for categoria, columnas in cat.items():
    # Sumar las columnas que corresponden a la categoría
    df_grouped[categoria] = df_formacio_clean[columnas].sum(axis=1)

# Afegim una nova columna amb el total per comarca, sumant les columnes anteriors
df_grouped['total'] = df_grouped.sum(axis=1)  # Sumar las columnas para obtener el total por comarca

# Mostrar el DataFrame resultante con las nuevas columnas de categorías
df_grouped.head()

,analfabetisme_total,educacio primaria incompleta_total,educacio primaria_total,educacio secundaria_total,formacio professional_total,graus universitaris_total,masters universitaris_total,doctorat universitari_total,total
0,120,1540,4530,21440,3785,5145,565,185,37310
1,385,6630,15960,70475,6590,16015,1505,575,118135
2,220,2365,12485,49395,8355,14545,1430,540,89335
3,50,805,2120,9820,1415,2840,360,110,17520
4,10,155,405,1715,275,700,65,25,3350


In [274]:
# Merge entre df_grouped i df_formacio_clean per afegir la columna 'comarca' de df_formacio_clean
df_formacio_clean = df_grouped.merge(df_formacio_clean[['comarca']], left_index=True, right_index=True, how='left')
df_formacio_clean.head()

,analfabetisme_total,educacio primaria incompleta_total,educacio primaria_total,educacio secundaria_total,formacio professional_total,graus universitaris_total,masters universitaris_total,doctorat universitari_total,total,comarca
0,120,1540,4530,21440,3785,5145,565,185,37310,alt camp
1,385,6630,15960,70475,6590,16015,1505,575,118135,alt emporda
2,220,2365,12485,49395,8355,14545,1430,540,89335,alt penedes
3,50,805,2120,9820,1415,2840,360,110,17520,alt urgell
4,10,155,405,1715,275,700,65,25,3350,alta ribagorca


In [276]:
# Reordenem les columnes per posar 'comarca' al principi
df_formacio_clean = df_formacio_clean[['comarca'] + [col for col in df_grouped.columns if col != 'comarca']]
df_formacio_clean.head()

,comarca,analfabetisme_total,educacio primaria incompleta_total,educacio primaria_total,educacio secundaria_total,formacio professional_total,graus universitaris_total,masters universitaris_total,doctorat universitari_total,total
0,alt camp,120,1540,4530,21440,3785,5145,565,185,37310
1,alt emporda,385,6630,15960,70475,6590,16015,1505,575,118135
2,alt penedes,220,2365,12485,49395,8355,14545,1430,540,89335
3,alt urgell,50,805,2120,9820,1415,2840,360,110,17520
4,alta ribagorca,10,155,405,1715,275,700,65,25,3350


In [278]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_formacio_clean.insert(0, 'codi', df_formacio_clean['comarca'].map(comarques_dict))
df_formacio_clean = df_formacio_clean.sort_values(by='codi').reset_index(drop=True)
df_formacio_clean.head()

,codi,comarca,analfabetisme_total,educacio primaria incompleta_total,educacio primaria_total,educacio secundaria_total,formacio professional_total,graus universitaris_total,masters universitaris_total,doctorat universitari_total,total
0,01,alt camp,120,1540,4530,21440,3785,5145,565,185,37310
1,02,alt emporda,385,6630,15960,70475,6590,16015,1505,575,118135
2,03,alt penedes,220,2365,12485,49395,8355,14545,1430,540,89335
3,04,alt urgell,50,805,2120,9820,1415,2840,360,110,17520
4,05,alta ribagorca,10,155,405,1715,275,700,65,25,3350


In [280]:
df_formacio_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   codi                                42 non-null     object
 1   comarca                             42 non-null     object
 2   analfabetisme_total                 42 non-null     int64 
 3   educacio primaria incompleta_total  42 non-null     int64 
 4   educacio primaria_total             42 non-null     int64 
 5   educacio secundaria_total           42 non-null     int64 
 6   formacio professional_total         42 non-null     int64 
 7   graus universitaris_total           42 non-null     int64 
 8   masters universitaris_total         42 non-null     int64 
 9   doctorat universitari_total         42 non-null     int64 
 10  total                               42 non-null     int64 
dtypes: int64(9), object(2)
memory usage: 3.7+ KB


In [282]:
df_formacio_clean.to_csv('clean_data/2018/df_formacio.csv', index=False)

#### Formació d'Adults<a class="anchor" id="formacio_adults"></a>

In [34]:
df_formacio_adults = pd.read_csv('csv/2018/formacio_adults_2018.csv', delimiter=';', skiprows=5)

In [35]:
df_formacio_adults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   comarca                    43 non-null     object
 1   Centres                    43 non-null     int64 
 2   Personal docent            43 non-null     int64 
 3   Alumnes (homes). 16-24     43 non-null     int64 
 4   Alumnes (homes). 25-64     43 non-null     int64 
 5   Alumnes (homes). 65 i més  43 non-null     int64 
 6   Alumnes (homes). Total     43 non-null     int64 
 7   Alumnes (dones). 16-24     43 non-null     int64 
 8   Alumnes (dones). 25-64     43 non-null     int64 
 9   Alumnes (dones). 65 i més  43 non-null     int64 
 10  Alumnes (dones). Total     43 non-null     int64 
 11  Alumnes (total). 16-24     43 non-null     int64 
 12  Alumnes (total). 25-64     43 non-null     int64 
 13  Alumnes (total). 65 i més  43 non-null     int64 
 14  Alumnes (tot

In [36]:
df_formacio_adults.head(10)

,comarca,Centres,Personal docent,Alumnes (homes). 16-24,Alumnes (homes). 25-64,Alumnes (homes). 65 i més,Alumnes (homes). Total,Alumnes (dones). 16-24,Alumnes (dones). 25-64,Alumnes (dones). 65 i més,Alumnes (dones). Total,Alumnes (total). 16-24,Alumnes (total). 25-64,Alumnes (total). 65 i més,Alumnes (total). Total
0,Alt Camp,1,7,32,42,12,86,33,137,51,221,65,179,63,307
1,Alt Empordà,4,33,144,513,14,671,81,196,16,293,225,709,30,964
2,Alt Penedès,3,16,53,38,19,110,41,73,20,134,94,111,39,244
3,Alt Urgell,1,6,13,63,19,95,21,204,29,254,34,267,48,349
4,Alta Ribagorça,1,1,5,3,0,8,3,15,0,18,8,18,0,26
5,Anoia,1,6,36,35,3,74,24,45,4,73,60,80,7,147
6,Aran,1,3,4,41,1,46,5,107,8,120,9,148,9,166
7,Bages,3,32,89,429,30,548,93,403,45,541,182,832,75,1089
8,Baix Camp,4,39,184,287,39,510,143,481,60,684,327,768,99,1194
9,Baix Ebre,2,16,61,89,28,178,51,209,36,296,112,298,64,474


In [37]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_formacio_adults_clean = clean_dataframe(df_formacio_adults)
df_formacio_adults_clean.head()

,comarca,centres,personal_docent,alumnes_(homes)._16-24,alumnes_(homes)._25-64,alumnes_(homes)._65_i_mes,alumnes_(homes)._total,alumnes_(dones)._16-24,alumnes_(dones)._25-64,alumnes_(dones)._65_i_mes,alumnes_(dones)._total,alumnes_(total)._16-24,alumnes_(total)._25-64,alumnes_(total)._65_i_mes,alumnes_(total)._total
0,alt camp,1,7,32,42,12,86,33,137,51,221,65,179,63,307
1,alt emporda,4,33,144,513,14,671,81,196,16,293,225,709,30,964
2,alt penedes,3,16,53,38,19,110,41,73,20,134,94,111,39,244
3,alt urgell,1,6,13,63,19,95,21,204,29,254,34,267,48,349
4,alta ribagorca,1,1,5,3,0,8,3,15,0,18,8,18,0,26


In [38]:
# Filtrar per comarques
df_formacio_adults_clean = filtrar_comarques(df_formacio_adults_clean, 'comarca')
df_formacio_adults_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   comarca                    42 non-null     object
 1   centres                    42 non-null     int64 
 2   personal_docent            42 non-null     int64 
 3   alumnes_(homes)._16-24     42 non-null     int64 
 4   alumnes_(homes)._25-64     42 non-null     int64 
 5   alumnes_(homes)._65_i_mes  42 non-null     int64 
 6   alumnes_(homes)._total     42 non-null     int64 
 7   alumnes_(dones)._16-24     42 non-null     int64 
 8   alumnes_(dones)._25-64     42 non-null     int64 
 9   alumnes_(dones)._65_i_mes  42 non-null     int64 
 10  alumnes_(dones)._total     42 non-null     int64 
 11  alumnes_(total)._16-24     42 non-null     int64 
 12  alumnes_(total)._25-64     42 non-null     int64 
 13  alumnes_(total)._65_i_mes  42 non-null     int64 
 14  alumnes_(tot

In [39]:
# Suma d'alumnes majors de 25 per a homes, dones i el total
df_formacio_adults_clean.loc[:, 'alumnes_homes_+25_anys'] = df_formacio_adults_clean['alumnes_(homes)._25-64'] + df_formacio_adults['alumnes_(homes)._65_i_mes']
df_formacio_adults_clean.loc[:, 'alumnes_dones_+25_anys'] = df_formacio_adults_clean['alumnes_(dones)._25-64'] + df_formacio_adults['alumnes_(dones)._65_i_mes']
df_formacio_adults_clean.loc[:, 'alumnes_total_+25_anys'] = df_formacio_adults_clean['alumnes_(total)._25-64'] + df_formacio_adults['alumnes_(total)._65_i_mes']


# Convertir les columnes +25 anys a tipus int
df_formacio_adults_clean[['alumnes_homes_+25_anys', 'alumnes_dones_+25_anys', 'alumnes_total_+25_anys']] = (
    df_formacio_adults_clean[['alumnes_homes_+25_anys', 'alumnes_dones_+25_anys', 'alumnes_total_+25_anys']].astype(int)
)

# Seleccionar i renombrar columnes
df_formacio_adults_clean = df_formacio_adults_clean[['comarca',
                                         'alumnes_(homes)._16-24',
                                         'alumnes_(dones)._16-24',
                                         'alumnes_(total)._16-24',
                                         'alumnes_homes_+25_anys',
                                         'alumnes_dones_+25_anys',
                                         'alumnes_total_+25_anys']].rename(columns={
    'alumnes_(homes)._16-24': 'alumnes_homes_16_24_anys',
    'alumnes_(dones)._16-24': 'alumnes_dones_16_24_anys',
    'alumnes_(total)._16-24': 'alumnes_total_16_24_anys'
})
# Dataset resultant
df_formacio_adults_clean.head()


,comarca,alumnes_homes_16_24_anys,alumnes_dones_16_24_anys,alumnes_total_16_24_anys,alumnes_homes_+25_anys,alumnes_dones_+25_anys,alumnes_total_+25_anys
0,alt camp,32,33,65,54,188,242
1,alt emporda,144,81,225,527,212,739
2,alt penedes,53,41,94,57,93,150
3,alt urgell,13,21,34,82,233,315
4,alta ribagorca,5,3,8,3,15,18


In [40]:
df_formacio_adults_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   comarca                   42 non-null     object
 1   alumnes_homes_16_24_anys  42 non-null     int64 
 2   alumnes_dones_16_24_anys  42 non-null     int64 
 3   alumnes_total_16_24_anys  42 non-null     int64 
 4   alumnes_homes_+25_anys    42 non-null     int32 
 5   alumnes_dones_+25_anys    42 non-null     int32 
 6   alumnes_total_+25_anys    42 non-null     int32 
dtypes: int32(3), int64(3), object(1)
memory usage: 1.9+ KB


In [41]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_formacio_adults_clean.insert(0, 'codi', df_formacio_adults_clean['comarca'].map(comarques_dict))
df_formacio_adults_clean = df_formacio_adults_clean.sort_values(by='codi').reset_index(drop=True)
df_formacio_adults_clean.head()

,codi,comarca,alumnes_homes_16_24_anys,alumnes_dones_16_24_anys,alumnes_total_16_24_anys,alumnes_homes_+25_anys,alumnes_dones_+25_anys,alumnes_total_+25_anys
0,01,alt camp,32,33,65,54,188,242
1,02,alt emporda,144,81,225,527,212,739
2,03,alt penedes,53,41,94,57,93,150
3,04,alt urgell,13,21,34,82,233,315
4,05,alta ribagorca,5,3,8,3,15,18


In [42]:
df_formacio_adults_clean.to_csv('clean_data/2018/df_formacio_adults.csv', index=False)

### Dades Població<a class="anchor" id="poblacio"></a>

#### Població per Gènere<a class="anchor" id="poblacio_genere"></a>

In [286]:
df_poblacio_genere = pd.read_csv('csv/2018/poblacio_genere_2018.csv', delimiter=';') 

In [288]:
df_poblacio_genere.head()

,comarca,dones,homes,total
0,Alt Camp,21794,22258,44052
1,Alt Empordà,69273,69769,139042
2,Alt Penedès,53852,53670,107522
3,Alt Urgell,10128,10190,20318
4,Alta Ribagorça,1843,1961,3804


In [290]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_poblacio_genere_clean = clean_dataframe(df_poblacio_genere)
df_poblacio_genere_clean.head()

,comarca,dones,homes,total
0,alt camp,21794,22258,44052
1,alt emporda,69273,69769,139042
2,alt penedes,53852,53670,107522
3,alt urgell,10128,10190,20318
4,alta ribagorca,1843,1961,3804


In [292]:
df_poblacio_genere_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'catalunya', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [294]:
# Filtrem per comarques
df_poblacio_genere_clean = filtrar_comarques(df_poblacio_genere_clean, 'comarca')

In [296]:
df_poblacio_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comarca  42 non-null     object
 1   dones    42 non-null     int64 
 2   homes    42 non-null     int64 
 3   total    42 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.4+ KB


In [298]:
df_poblacio_genere_clean.head()

,comarca,dones,homes,total
0,alt camp,21794,22258,44052
1,alt emporda,69273,69769,139042
2,alt penedes,53852,53670,107522
3,alt urgell,10128,10190,20318
4,alta ribagorca,1843,1961,3804


In [300]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_genere_clean.insert(0, 'codi', df_poblacio_genere_clean['comarca'].map(comarques_dict))
df_poblacio_genere_clean = df_poblacio_genere_clean.sort_values(by='codi').reset_index(drop=True)
df_poblacio_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   codi     42 non-null     object
 1   comarca  42 non-null     object
 2   dones    42 non-null     int64 
 3   homes    42 non-null     int64 
 4   total    42 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 1.8+ KB


In [302]:
df_poblacio_genere_clean.head()

,codi,comarca,dones,homes,total
0,01,alt camp,21794,22258,44052
1,02,alt emporda,69273,69769,139042
2,03,alt penedes,53852,53670,107522
3,04,alt urgell,10128,10190,20318
4,05,alta ribagorca,1843,1961,3804


In [304]:
df_poblacio_genere_clean.to_csv('clean_data/2018/df_poblacio_genere.csv', index=False)

#### Població per Edat<a class="anchor" id="poblacio_edat"></a>

In [306]:
df_poblacio_edat = pd.read_csv('csv/2018/poblacio_edat_2018.csv', delimiter=';')

In [308]:
df_poblacio_edat.head()

,comarca,poblacio_total_16_a_24_anys,poblacio_homes_16_a_24_anys,poblacio_dones_16_a_24_anys
0,Alt Camp,3941,2013,1928
1,Alt Empordà,12458,6455,6003
2,Alt Penedès,9375,4738,4637
3,Alt Urgell,1754,913,841
4,Alta Ribagorça,300,150,150


In [310]:
df_poblacio_edat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   comarca                      55 non-null     object
 1   poblacio_total_16_a_24_anys  55 non-null     int64 
 2   poblacio_homes_16_a_24_anys  55 non-null     int64 
 3   poblacio_dones_16_a_24_anys  55 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.8+ KB


In [312]:
# Netejar el DataFrame 'df_poblacio_edat' utilitzant la funció 'clean_dataframe'
df_poblacio_edat_clean = clean_dataframe(df_poblacio_edat)
df_poblacio_edat_clean.head()

,comarca,poblacio_total_16_a_24_anys,poblacio_homes_16_a_24_anys,poblacio_dones_16_a_24_anys
0,alt camp,3941,2013,1928
1,alt emporda,12458,6455,6003
2,alt penedes,9375,4738,4637
3,alt urgell,1754,913,841
4,alta ribagorca,300,150,150


In [314]:
# Filtrar les comarques utilitzant la funció 'filtrar_comarques'
df_poblacio_edat_clean = filtrar_comarques(df_poblacio_edat_clean, 'comarca')
df_poblacio_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   comarca                      42 non-null     object
 1   poblacio_total_16_a_24_anys  42 non-null     int64 
 2   poblacio_homes_16_a_24_anys  42 non-null     int64 
 3   poblacio_dones_16_a_24_anys  42 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.4+ KB


In [316]:
df_poblacio_edat_clean.head()

,comarca,poblacio_total_16_a_24_anys,poblacio_homes_16_a_24_anys,poblacio_dones_16_a_24_anys
0,alt camp,3941,2013,1928
1,alt emporda,12458,6455,6003
2,alt penedes,9375,4738,4637
3,alt urgell,1754,913,841
4,alta ribagorca,300,150,150


In [318]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_edat_clean.insert(0, 'codi', df_poblacio_edat_clean['comarca'].map(comarques_dict))
df_poblacio_edat_clean = df_poblacio_edat_clean.sort_values(by='codi').reset_index(drop=True)
df_poblacio_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   codi                         42 non-null     object
 1   comarca                      42 non-null     object
 2   poblacio_total_16_a_24_anys  42 non-null     int64 
 3   poblacio_homes_16_a_24_anys  42 non-null     int64 
 4   poblacio_dones_16_a_24_anys  42 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 1.8+ KB


In [320]:
df_poblacio_edat_clean.head()

,codi,comarca,poblacio_total_16_a_24_anys,poblacio_homes_16_a_24_anys,poblacio_dones_16_a_24_anys
0,01,alt camp,3941,2013,1928
1,02,alt emporda,12458,6455,6003
2,03,alt penedes,9375,4738,4637
3,04,alt urgell,1754,913,841
4,05,alta ribagorca,300,150,150


In [322]:
df_poblacio_edat_clean.to_csv('clean_data/2018/df_poblacio_edat.csv', index=False)

#### Atur per Edat<a class="anchor" id="atur"></a>

In [100]:
df_atur_edat = pd.read_csv('csv/2018/atur_edat_2018.csv', delimiter=';', skiprows=6)

In [101]:
df_atur_edat.head()

,comarca,16-24,25+,Total
0,Alt Camp,201.8,2392.2,2594.0
1,Alt Empordà,577.1,7069.8,7646.9
2,Alt Penedès,313.1,5043.1,5356.2
3,Alt Urgell,60.2,738.3,798.5
4,Alta Ribagorça,3.7,87.0,90.7


In [102]:
df_atur_edat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comarca  55 non-null     object 
 1   16-24    55 non-null     float64
 2   25+      55 non-null     float64
 3   Total    55 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.8+ KB


In [103]:
# Netejar el DataFrame 'df_atur_edat' amb la funció 'clean_dataframe'
df_atur_edat_clean = clean_dataframe(df_atur_edat)
df_atur_edat_clean.head()

,comarca,16-24,25+,total
0,alt camp,201.8,2392.2,2594.0
1,alt emporda,577.1,7069.8,7646.9
2,alt penedes,313.1,5043.1,5356.2
3,alt urgell,60.2,738.3,798.5
4,alta ribagorca,3.7,87.0,90.7


In [104]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_atur_edat_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'cataluna', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [105]:
# Filtrar les comarques en el DataFrame netejat
df_atur_edat_clean = filtrar_comarques(df_atur_edat_clean, 'comarca')
# Obtenir el nombre total de comarques úniques després del filtratge
len(df_atur_edat_clean['comarca'].unique())

42

In [106]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_atur_edat_clean.insert(0, 'codi', df_atur_edat_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_atur_edat_clean = df_atur_edat_clean.sort_values(by='codi').reset_index(drop=True)
df_atur_edat_clean.head()

,codi,comarca,16-24,25+,total
0,01,alt camp,201.8,2392.2,2594.0
1,02,alt emporda,577.1,7069.8,7646.9
2,03,alt penedes,313.1,5043.1,5356.2
3,04,alt urgell,60.2,738.3,798.5
4,05,alta ribagorca,3.7,87.0,90.7


In [107]:
df_atur_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   codi     42 non-null     object 
 1   comarca  42 non-null     object 
 2   16-24    42 non-null     float64
 3   25+      42 non-null     float64
 4   total    42 non-null     float64
dtypes: float64(3), object(2)
memory usage: 1.8+ KB


In [108]:
# Identificar les columnes de tipus float al dataset
columnes_float = df_atur_edat_clean.select_dtypes(include=['float']).columns

# Convertir les columnes de float a int
df_atur_edat_clean[columnes_float] = (
    np.ceil(df_atur_edat_clean[columnes_float]).astype(int)
)
# Verificar el resultat
df_atur_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   codi     42 non-null     object
 1   comarca  42 non-null     object
 2   16-24    42 non-null     int32 
 3   25+      42 non-null     int32 
 4   total    42 non-null     int32 
dtypes: int32(3), object(2)
memory usage: 1.3+ KB


In [109]:
df_atur_edat_clean.head()

,codi,comarca,16-24,25+,total
0,01,alt camp,202,2393,2594
1,02,alt emporda,578,7070,7647
2,03,alt penedes,314,5044,5357
3,04,alt urgell,61,739,799
4,05,alta ribagorca,4,87,91


In [110]:
df_atur_edat_clean.to_csv('clean_data/2018/df_atur_edat.csv', index=False)

#### Atur per Gènere<a class="anchor" id="atur_genere"></a>

In [328]:
df_atur_genere = pd.read_csv('csv/2018/atur_genere_2018.csv', delimiter=';', skiprows=6)

In [330]:
df_atur_genere.head()

,comarca,Total_homes,Total_dones,Total
0,Alt Camp,1132.3,1461.5,2593.8
1,Alt Empordà,3436.9,4209.9,7646.8
2,Alt Penedès,2239.4,3116.8,5356.2
3,Alt Urgell,377.3,421.0,798.3
4,Alta Ribagorça,48.3,42.2,90.5


In [332]:
df_atur_genere.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   comarca      55 non-null     object 
 1   Total_homes  55 non-null     float64
 2   Total_dones  55 non-null     float64
 3   Total        55 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.8+ KB


In [334]:
# Netejar el DataFrame 'df_atur_genere' amb la funció 'clean_dataframe'
df_atur_genere_clean = clean_dataframe(df_atur_genere)
df_atur_genere_clean.head()

,comarca,total_homes,total_dones,total
0,alt camp,1132.3,1461.5,2593.8
1,alt emporda,3436.9,4209.9,7646.8
2,alt penedes,2239.4,3116.8,5356.2
3,alt urgell,377.3,421.0,798.3
4,alta ribagorca,48.3,42.2,90.5


In [336]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_atur_genere_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'cataluna', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [338]:
# Filtrar les comarques en el DataFrame netejat
df_atur_genere_clean = filtrar_comarques(df_atur_genere_clean, 'comarca')
# Obtenir el nombre total de comarques úniques després del filtratge
len(df_atur_genere_clean['comarca'].unique())

42

In [340]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_atur_genere_clean.insert(0, 'codi', df_atur_genere_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_atur_genere_clean = df_atur_genere_clean.sort_values(by='codi').reset_index(drop=True)
df_atur_genere_clean.head()

,codi,comarca,total_homes,total_dones,total
0,01,alt camp,1132.3,1461.5,2593.8
1,02,alt emporda,3436.9,4209.9,7646.8
2,03,alt penedes,2239.4,3116.8,5356.2
3,04,alt urgell,377.3,421.0,798.3
4,05,alta ribagorca,48.3,42.2,90.5


In [342]:
df_atur_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   codi         42 non-null     object 
 1   comarca      42 non-null     object 
 2   total_homes  42 non-null     float64
 3   total_dones  42 non-null     float64
 4   total        42 non-null     float64
dtypes: float64(3), object(2)
memory usage: 1.8+ KB


In [344]:
# Identificar les columnes de tipus float al dataset
columnes_float = df_atur_genere_clean.select_dtypes(include=['float']).columns

# Convertir les columnes de float a int
df_atur_genere_clean[columnes_float] = (
    np.ceil(df_atur_genere_clean[columnes_float]).astype(int)
)
# Verificar el resultat
df_atur_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   codi         42 non-null     object
 1   comarca      42 non-null     object
 2   total_homes  42 non-null     int32 
 3   total_dones  42 non-null     int32 
 4   total        42 non-null     int32 
dtypes: int32(3), object(2)
memory usage: 1.3+ KB


In [346]:
df_atur_genere_clean.head()

,codi,comarca,total_homes,total_dones,total
0,01,alt camp,1133,1462,2594
1,02,alt emporda,3437,4210,7647
2,03,alt penedes,2240,3117,5357
3,04,alt urgell,378,421,799
4,05,alta ribagorca,49,43,91


In [348]:
df_atur_genere_clean.to_csv('clean_data/2018/df_atur_genere.csv', index=False)

#### Població per Nacionalitat<a class="anchor" id="poblacio_nacionalitat"></a>

In [162]:
df_poblacio_nacionalitat = pd.read_csv('csv/2018/poblacio_nacionalitat_2018.csv', delimiter=';', skiprows=6)

In [163]:
df_poblacio_nacionalitat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   comarca                     55 non-null     object
 1   Espanyola                   55 non-null     int64 
 2   Resta UE                    55 non-null     int64 
 3   Resta d'Europa              55 non-null     int64 
 4   Àfrica                      55 non-null     int64 
 5   Amèrica del Nord i Central  55 non-null     int64 
 6   Amèrica del Sud             55 non-null     int64 
 7   Àsia i Oceania              55 non-null     int64 
 8   Total                       55 non-null     int64 
dtypes: int64(8), object(1)
memory usage: 4.0+ KB


In [164]:
df_poblacio_nacionalitat.head()

,comarca,Espanyola,Resta UE,Resta d'Europa,Àfrica,Amèrica del Nord i Central,Amèrica del Sud,Àsia i Oceania,Total
0,Alt Camp,38948,1214,448,2609,165,474,207,44065
1,Alt Empordà,106913,12101,2045,12760,1960,3758,1032,140569
2,Alt Penedès,96818,1884,616,5987,505,1635,677,108122
3,Alt Urgell,17958,1052,217,333,106,446,112,20224
4,Alta Ribagorça,3364,176,55,98,14,61,6,3774


In [165]:
# Netejar el DataFrame 'df_poblacio_nacionalitat' amb la funció 'clean_dataframe'
df_poblacio_nacionalitat_clean = clean_dataframe(df_poblacio_nacionalitat)
# Filtrar les comarques en el DataFrame netejat
df_poblacio_nacionalitat_clean = filtrar_comarques(df_poblacio_nacionalitat_clean, 'comarca')
df_poblacio_nacionalitat_clean.head()

,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total
0,alt camp,38948,1214,448,2609,165,474,207,44065
1,alt emporda,106913,12101,2045,12760,1960,3758,1032,140569
2,alt penedes,96818,1884,616,5987,505,1635,677,108122
3,alt urgell,17958,1052,217,333,106,446,112,20224
4,alta ribagorca,3364,176,55,98,14,61,6,3774


In [166]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_nacionalitat_clean.insert(0, 'codi', df_poblacio_nacionalitat_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_poblacio_nacionalitat_clean = df_poblacio_nacionalitat_clean.sort_values(by='codi').reset_index(drop=True)
df_poblacio_nacionalitat_clean.head()

,codi,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total
0,01,alt camp,38948,1214,448,2609,165,474,207,44065
1,02,alt emporda,106913,12101,2045,12760,1960,3758,1032,140569
2,03,alt penedes,96818,1884,616,5987,505,1635,677,108122
3,04,alt urgell,17958,1052,217,333,106,446,112,20224
4,05,alta ribagorca,3364,176,55,98,14,61,6,3774


In [167]:
# Llista de nacionalitats
nacionalitats = ['espanyola', 'resta_ue', 'resta_d\'europa', 'africa', 'america_del_nord_i_central', 'america_del_sud', 'asia_i_oceania']

# Calcular el percentatge i la ràtio per a cada nacionalitat
for nacionalitat in nacionalitats:
    # Percentatge respecte al total per comarca
    df_poblacio_nacionalitat_clean[f'{nacionalitat}_pct'] = (
        df_poblacio_nacionalitat_clean[nacionalitat] / df_poblacio_nacionalitat_clean['total'] * 100
    ).round(2)
    
    # Ràtio respecte al total per comarca
    df_poblacio_nacionalitat_clean[f'{nacionalitat}_ratio'] = (
        df_poblacio_nacionalitat_clean[nacionalitat] / df_poblacio_nacionalitat_clean['total']
    )

# Mostrar les primeres files del DataFrame resultant
df_poblacio_nacionalitat_clean.head()

,codi,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total,...,resta_d'europa_pct,resta_d'europa_ratio,africa_pct,africa_ratio,america_del_nord_i_central_pct,america_del_nord_i_central_ratio,america_del_sud_pct,america_del_sud_ratio,asia_i_oceania_pct,asia_i_oceania_ratio
0,01,alt camp,38948,1214,448,2609,165,474,207,44065,...,1.02,0.010167,5.92,0.059208,0.37,0.003744,1.08,0.010757,0.47,0.004698
1,02,alt emporda,106913,12101,2045,12760,1960,3758,1032,140569,...,1.45,0.014548,9.08,0.090774,1.39,0.013943,2.67,0.026734,0.73,0.007342
2,03,alt penedes,96818,1884,616,5987,505,1635,677,108122,...,0.57,0.005697,5.54,0.055373,0.47,0.004671,1.51,0.015122,0.63,0.006261
3,04,alt urgell,17958,1052,217,333,106,446,112,20224,...,1.07,0.010730,1.65,0.016466,0.52,0.005241,2.21,0.022053,0.55,0.005538
4,05,alta ribagorca,3364,176,55,98,14,61,6,3774,...,1.46,0.014573,2.60,0.025967,0.37,0.003710,1.62,0.016163,0.16,0.001590


In [168]:
df_poblacio_nacionalitat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   codi                              42 non-null     object 
 1   comarca                           42 non-null     object 
 2   espanyola                         42 non-null     int64  
 3   resta_ue                          42 non-null     int64  
 4   resta_d'europa                    42 non-null     int64  
 5   africa                            42 non-null     int64  
 6   america_del_nord_i_central        42 non-null     int64  
 7   america_del_sud                   42 non-null     int64  
 8   asia_i_oceania                    42 non-null     int64  
 9   total                             42 non-null     int64  
 10  espanyola_pct                     42 non-null     float64
 11  espanyola_ratio                   42 non-null     float64
 12  resta_ue_p

In [169]:
df_poblacio_nacionalitat_clean.to_csv('clean_data/2018/df_poblacio_nacionalitat.csv', index=False)

### Dades Socioeconòmiques<a class="anchor" id="dades_socioeconomiques"></a>

#### Repetidors<a class="anchor" id="repetidors"></a>

https://educacio.gencat.cat/ca/departament/estadistiques/estadistiques-ensenyament/cursos-anteriors/curs-2021-2022/eso/

In [173]:
df_repetidors = pd.read_csv('csv/2018/alumnes_repetidors_2018.csv', delimiter=';')

In [174]:
df_repetidors.head()

,Ârea territorial codi,Àrea territorial,Comarca codi,Comarca,Municipi codi,Municipi,Districte codi,Naturalesa,Titularitat,Estudis,Nivell,Sexe,Concert,Resultat de l'avaluació,N Avaluats estudi
0,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,SI,Alumnes avaluats,3
1,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,SI,No promocionen,1
2,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,SI,Promocionen al mes de juny,1
3,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,SI,Promocionen amb pendents,1
4,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,HOME,SI,Alumnes avaluats,9


In [175]:
df_repetidors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21003 entries, 0 to 21002
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Ârea territorial codi    21003 non-null  int64 
 1   Àrea territorial         21003 non-null  object
 2   Comarca codi             21003 non-null  int64 
 3   Comarca                  21003 non-null  object
 4   Municipi codi            21003 non-null  int64 
 5   Municipi                 21003 non-null  object
 6   Districte codi           21003 non-null  int64 
 7   Naturalesa               21003 non-null  object
 8   Titularitat              21003 non-null  object
 9   Estudis                  21003 non-null  object
 10  Nivell                   21003 non-null  int64 
 11  Sexe                     21003 non-null  object
 12  Concert                  9829 non-null   object
 13  Resultat de l'avaluació  21003 non-null  object
 14  N Avaluats estudi        21003 non-nul

In [176]:
df_repetidors['Comarca'].unique()

array(['BARCELONÈS', 'ALT EMPORDÀ', 'BAIX EMPORDÀ', 'CERDANYA',
       'GARROTXA', 'GIRONÈS', "PLA DE L'ESTANY", 'RIPOLLÈS', 'SELVA',
       'ALT URGELL', 'ALTA RIBAGORÇA', 'GARRIGUES', 'NOGUERA',
       'PALLARS JUSSÀ', 'PALLARS SOBIRÀ', "PLA D'URGELL", 'SEGARRA',
       'SEGRIÀ', 'URGELL', "VAL D'ARAN", 'ALT CAMP', 'BAIX CAMP',
       'BAIX PENEDÈS', 'CONCA DE BARBERÀ', 'PRIORAT', 'TARRAGONÈS',
       'ALT PENEDÈS', 'GARRAF', 'BAIX EBRE', 'MONTSIÀ', "RIBERA D'EBRE",
       'TERRA ALTA', 'BAIX LLOBREGAT', 'VALLÈS OCCIDENTAL', 'MARESME',
       'VALLÈS ORIENTAL', 'ANOIA', 'BAGES', 'BERGUEDÀ', 'OSONA',
       'SOLSONÈS', 'MOIANÈS'], dtype=object)

In [177]:
# Netejar el DataFrame 'df_repetidors' utilitzant la funció 'clean_dataframe'
df_repetidors_clean = clean_dataframe(df_repetidors)
df_repetidors_clean.head()

,area_territorial_codi,area_territorial,comarca_codi,comarca,municipi_codi,municipi,districte_codi,naturalesa,titularitat,estudis,nivell,sexe,concert,resultat_de_l'avaluacio,n_avaluats_estudi
0,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,associacions,educacio secundaria obligatoria,1,dona,si,alumnes avaluats,3
1,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,associacions,educacio secundaria obligatoria,1,dona,si,no promocionen,1
2,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,associacions,educacio secundaria obligatoria,1,dona,si,promocionen al mes de juny,1
3,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,associacions,educacio secundaria obligatoria,1,dona,si,promocionen amb pendents,1
4,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,associacions,educacio secundaria obligatoria,1,home,si,alumnes avaluats,9


In [178]:
# Filtrar les dades del DataFrame per obtenir només les comarques rellevants
df_repetidors_clean = filtrar_comarques(df_repetidors_clean, 'comarca')

In [179]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
display(df_repetidors_clean['comarca'].unique())
print(len(df_repetidors_clean['comarca'].unique()))

array(['barcelones', 'alt emporda', 'baix emporda', 'cerdanya',
       'garrotxa', 'girones', "pla de l'estany", 'ripolles', 'selva',
       'alt urgell', 'alta ribagorca', 'garrigues', 'noguera',
       'pallars jussa', 'pallars sobira', "pla d'urgell", 'segarra',
       'segria', 'urgell', 'alt camp', 'baix camp', 'baix penedes',
       'conca de barbera', 'priorat', 'tarragones', 'alt penedes',
       'garraf', 'baix ebre', 'montsia', "ribera d'ebre", 'terra alta',
       'baix llobregat', 'valles occidental', 'maresme',
       'valles oriental', 'anoia', 'bages', 'bergueda', 'osona',
       'solsones', 'moianes'], dtype=object)

41


In [180]:
# Obtenir els valors únics de la columna 'resultat_de_l'avaluacio' en el DataFrame netejat
df_repetidors_clean['resultat_de_l\'avaluacio'].unique()

array(['alumnes avaluats', 'no promocionen', 'promocionen al mes de juny',
       'promocionen amb pendents', 'promocionen al mes de setembre',
       "s'ha expedit certificat"], dtype=object)

In [181]:
# Filtrar el DataFrame per mantenir només les files amb 'resultat_de_l'avaluacio' igual a 'no promocionen'
df_no_promocionen = df_repetidors_clean[df_repetidors_clean['resultat_de_l\'avaluacio'] == 'no promocionen']

# Agrupar les dades per 'comarca' i 'sexe', sumant els valors de 'nombre_alumnes'
df_repetidors = df_no_promocionen.groupby(['comarca', 'sexe'])['n_avaluats_estudi'].sum().reset_index()

# Renombrar la columna 'nombre_alumnes' a 'repetixen'
df_repetidors = df_repetidors.rename(columns={'n_avaluats_estudi': 'repetixen'})

# Mostrar les primeres files del DataFrame resultant
df_repetidors.head()

,comarca,sexe,repetixen
0,alt camp,dona,45
1,alt camp,home,61
2,alt emporda,dona,125
3,alt emporda,home,161
4,alt penedes,dona,66


In [182]:
# Afegim la columna 'codi'  per fer match amb els noms de les comarques
df_repetidors.insert(0, 'codi', df_repetidors['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_repetidors = df_repetidors.sort_values(by='codi').reset_index(drop=True)
df_repetidors.head()

,codi,comarca,sexe,repetixen
0,01,alt camp,dona,45
1,01,alt camp,home,61
2,02,alt emporda,dona,125
3,02,alt emporda,home,161
4,03,alt penedes,dona,66


In [183]:
df_repetidors.to_csv('clean_data/2018/df_repetidors.csv', index=False)

In [184]:
df_repetidors.head()

,codi,comarca,sexe,repetixen
0,01,alt camp,dona,45
1,01,alt camp,home,61
2,02,alt emporda,dona,125
3,02,alt emporda,home,161
4,03,alt penedes,dona,66


In [185]:
# Agrupar les dades per 'codi' i 'comarca', sumant els valors de 'repetixen'
df_total_repetidors = df_repetidors.groupby(['codi', 'comarca'])['repetixen'].sum().reset_index()

# Renombrar la columna 'repetixen' a 'total_repetixen'
df_total_repetidors.rename(columns={'repetixen': 'total_repetixen'}, inplace=True)
df_total_repetidors.head()

,codi,comarca,total_repetixen
0,01,alt camp,106
1,02,alt emporda,286
2,03,alt penedes,185
3,04,alt urgell,23
4,05,alta ribagorca,8


In [186]:
df_pct_ratio_estudiants.head()

NameError: name 'df_pct_ratio_estudiants' is not defined

In [ ]:
# Fusionar els DataFrames 'df_total_repetidors' i 'df_pct_ratio_estudiants' utilitzant la columna comuna 'codi'
df_total_repetidors_merged = df_total_repetidors.merge(df_pct_ratio_estudiants[['codi', 'poblacio_estudiant']], on='codi', how='left')

# Seleccionar només les columnes necessàries del DataFrame resultant
df_total_repetidors_merged = df_total_repetidors_merged[['codi', 'comarca', 'total_repetixen', 'poblacio_estudiant']].copy()
df_total_repetidors_merged.head()

In [ ]:
# Calcular el percentatge de repetidors respecte a la població estudiant
df_total_repetidors_merged['pct_repetidors'] = (df_total_repetidors_merged['total_repetixen'] / df_total_repetidors_merged['poblacio_estudiant'] * 100).round(2)

# Calcular la ràtio de repetidors respecte a la població estudiant
df_total_repetidors_merged['ratio_repetidors'] = (df_total_repetidors_merged['total_repetixen'] / df_total_repetidors_merged['poblacio_estudiant'])
df_total_repetidors_merged.head()

In [ ]:
df_total_repetidors_merged.to_csv('clean_data/2018/df_total_repetidors.csv', index=False)

In [ ]:
df_repetidors.head()

In [ ]:
df_total_repetidors_merged.head()

In [ ]:
# Fusionar 'df_repetidors' amb 'df_total_repetidors_merged' per afegir 'comarca' i 'poblacio_estudiant'
df_repetidors_genere_merged = df_repetidors.merge(
    df_total_repetidors_merged[['codi', 'comarca', 'poblacio_estudiant']], 
    on='codi', 
    how='left'
)
df_repetidors_genere_merged.head()

In [ ]:
# Eliminar 'comarca_y' i renombrar 'comarca_x' a 'comarca'
df_repetidors_genere_merged = df_repetidors_genere_merged.drop(columns=['comarca_y']).rename(columns={'comarca_x': 'comarca'})

# Calcular el total de repetidors i el total d'estudiants per comarca
df_total_repetidors = df_repetidors_genere_merged.groupby(['codi', 'comarca'])[['repetixen', 'poblacio_estudiant']].sum().reset_index()
df_total_repetidors.rename(columns={'repetixen': 'total_repetixen', 'poblacio_estudiant': 'total_estudiants'}, inplace=True)

# Afegir els totals al DataFrame original i mantenir 'comarca'
df_repetidors_genere_merged = df_repetidors_genere_merged.merge(
    df_total_repetidors[['codi', 'comarca', 'total_repetixen', 'total_estudiants']], 
    on=['codi', 'comarca'], 
    how='left'
)

df_repetidors_genere_merged.head()

In [ ]:
# Calcular el percentatge de repetidors per dones
df_repetidors_genere_merged['pct_repetidors_dones'] = (
    df_repetidors_genere_merged.loc[df_repetidors_genere_merged['sexe'] == 'dona', 'repetixen'] /
    df_repetidors_genere_merged['total_estudiants'] * 100
).fillna(0).round(2)

# Calcular el percentatge de repetidors per homes
df_repetidors_genere_merged['pct_repetidors_homes'] = (
    df_repetidors_genere_merged.loc[df_repetidors_genere_merged['sexe'] == 'home', 'repetixen'] /
    df_repetidors_genere_merged['total_estudiants'] * 100
).fillna(0).round(2)

# Calcular la ràtio de repetidors per dones
df_repetidors_genere_merged['ratio_repetidors_dones'] = (
    df_repetidors_genere_merged.loc[df_repetidors_genere_merged['sexe'] == 'dona', 'repetixen'] /
    df_repetidors_genere_merged['total_estudiants']
).fillna(0)

# Calcular la ràtio de repetidors per homes
df_repetidors_genere_merged['ratio_repetidors_homes'] = (
    df_repetidors_genere_merged.loc[df_repetidors_genere_merged['sexe'] == 'home', 'repetixen'] /
    df_repetidors_genere_merged['total_estudiants']
).fillna(0)

# Seleccionar només les columnes necessàries i eliminar duplicats
df_repetidors_poblacio = df_repetidors_genere_merged[['codi', 'comarca', 'total_repetixen', 
                                         'pct_repetidors_dones', 'pct_repetidors_homes', 
                                         'ratio_repetidors_dones', 'ratio_repetidors_homes']].drop_duplicates()

# Mostrar el DataFrame resultant
df_repetidors_poblacio.head()


In [ ]:
# Agrupar per comarca i codi per sumar els valors de dones i homes
df_repetidors_poblacio = df_repetidors_poblacio.groupby(['codi', 'comarca'], as_index=False).sum()

df_repetidors_poblacio.head()

In [ ]:
df_repetidors_poblacio.to_csv('clean_data/2018/df_repetidors_poblacio.csv', index=False)

### Nivell Socioeconòmic (Renda Familiar)<a class="anchor" id="renta_familiar"></a>

In [203]:
df_renta_familiar = pd.read_csv('csv/2018/renta_familiar2018.csv', delimiter=';', skiprows=6)

In [204]:
df_renta_familiar.head()

,Unnamed: 0,RFDB (miles de euros),RFDB por habitante (euros),RFDB por habitante (Índex Catalunya=100 por habitante)
0,Alt Camp,688231,15537,"88,9"
1,Alt Empordà,1978509,14393,"82,3"
2,Alt Penedès,1765253,16340,"93,5"
3,Alt Urgell,276699,13713,"78,4"
4,Alta Ribagorça,57443,15051,"86,1"


In [205]:
# Renombrar la columna 'Unnamed: 0' a 'comarca'
df_renta_familiar = df_renta_familiar.rename(columns={'Unnamed: 0': 'comarca'})
df_renta_familiar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 4 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   comarca                                                 51 non-null     object
 1   RFDB (miles de euros)                                   51 non-null     int64 
 2   RFDB por habitante (euros)                              51 non-null     int64 
 3   RFDB por habitante (Índex Catalunya=100 por habitante)  51 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.7+ KB


In [206]:
# Netejar el DataFrame 'df_renta_familiar' utilitzant la funció 'clean_dataframe'
df_renta_familiar_clean = clean_dataframe(df_renta_familiar)
# Filtrar les dades per obtenir només les comarques rellevants
df_renta_familiar_clean = filtrar_comarques(df_renta_familiar_clean, 'comarca')
df_renta_familiar_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   comarca                                                 42 non-null     object
 1   rfdb_(miles_de_euros)                                   42 non-null     int64 
 2   rfdb_por_habitante_(euros)                              42 non-null     int64 
 3   rfdb_por_habitante_(index_catalunya=100_por_habitante)  42 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.4+ KB


In [207]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_renta_familiar_clean.insert(0, 'codi', df_renta_familiar_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_renta_familiar_clean = df_renta_familiar_clean.sort_values(by='codi').reset_index(drop=True)
df_renta_familiar_clean.head()

,codi,comarca,rfdb_(miles_de_euros),rfdb_por_habitante_(euros),rfdb_por_habitante_(index_catalunya=100_por_habitante)
0,01,alt camp,688231,15537,"88,9"
1,02,alt emporda,1978509,14393,"82,3"
2,03,alt penedes,1765253,16340,"93,5"
3,04,alt urgell,276699,13713,"78,4"
4,05,alta ribagorca,57443,15051,"86,1"


In [208]:
# Convertir els valors de la columna a format numèric
df_renta_familiar_clean['rfdb_por_habitante_(index_catalunya=100_por_habitante)'] = (
    df_renta_familiar_clean['rfdb_por_habitante_(index_catalunya=100_por_habitante)']
    .str.replace(',', '.')  # Substituir les comes per punts
    .astype(float)          # Convertir els valors a tipus float
)
df_renta_familiar_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   codi                                                    42 non-null     object 
 1   comarca                                                 42 non-null     object 
 2   rfdb_(miles_de_euros)                                   42 non-null     int64  
 3   rfdb_por_habitante_(euros)                              42 non-null     int64  
 4   rfdb_por_habitante_(index_catalunya=100_por_habitante)  42 non-null     float64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.8+ KB


In [209]:
df_renta_familiar_clean.head()

,codi,comarca,rfdb_(miles_de_euros),rfdb_por_habitante_(euros),rfdb_por_habitante_(index_catalunya=100_por_habitante)
0,01,alt camp,688231,15537,88.9
1,02,alt emporda,1978509,14393,82.3
2,03,alt penedes,1765253,16340,93.5
3,04,alt urgell,276699,13713,78.4
4,05,alta ribagorca,57443,15051,86.1


In [210]:
df_renta_familiar_clean.to_csv('clean_data/2018/df_renta_familiar.csv', index=False)

### Nivell Socioeconòmic (RFBD)<a class="anchor" id="rfbd"></a>

In [212]:
df_rfbd = pd.read_csv('csv/2018/RFBD_2018.csv', delimiter=';', skiprows=6)

In [213]:
df_rfbd.head()

,Unnamed: 0,Recursos. Remuneración asalariados,Recursos. Excedente bruto explotación,Recursos. Prestaciones sociales,Usos. Cotizaciones sociales,Usos. Impuestos
0,Alt Camp,544469,186056,191794,161595,89680
1,Alt Empordà,1449595,674267,450761,448546,295196
2,Alt Penedès,1449403,492449,471889,421447,255331
3,Alt Urgell,177640,97022,80091,56810,32616
4,Alta Ribagorça,38552,24013,14531,12735,8260


In [214]:
# Renombrar la columna 'Unnamed: 0' a 'comarca'
df_rfbd = df_rfbd.rename(columns={'Unnamed: 0': 'comarca'})
df_rfbd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   comarca                                51 non-null     object
 1   Recursos. Remuneración asalariados     51 non-null     int64 
 2   Recursos. Excedente bruto explotación  51 non-null     int64 
 3   Recursos. Prestaciones sociales        51 non-null     int64 
 4   Usos. Cotizaciones sociales            51 non-null     int64 
 5   Usos. Impuestos                        51 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.5+ KB


In [215]:
# Netejar el DataFrame 'df_renta_familiar' utilitzant la funció 'clean_dataframe'
df_rfbd_clean = clean_dataframe(df_rfbd)
# Filtrar les dades per obtenir només les comarques rellevants
df_rfbd_clean = filtrar_comarques(df_rfbd_clean, 'comarca')
df_rfbd_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   comarca                                42 non-null     object
 1   recursos._remuneracion_asalariados     42 non-null     int64 
 2   recursos._excedente_bruto_explotacion  42 non-null     int64 
 3   recursos._prestaciones_sociales        42 non-null     int64 
 4   usos._cotizaciones_sociales            42 non-null     int64 
 5   usos._impuestos                        42 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.1+ KB


In [216]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_rfbd_clean.insert(0, 'codi', df_rfbd_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_rfbd_clean = df_rfbd_clean.sort_values(by='codi').reset_index(drop=True)
df_rfbd_clean.head()

,codi,comarca,recursos._remuneracion_asalariados,recursos._excedente_bruto_explotacion,recursos._prestaciones_sociales,usos._cotizaciones_sociales,usos._impuestos
0,01,alt camp,544469,186056,191794,161595,89680
1,02,alt emporda,1449595,674267,450761,448546,295196
2,03,alt penedes,1449403,492449,471889,421447,255331
3,04,alt urgell,177640,97022,80091,56810,32616
4,05,alta ribagorca,38552,24013,14531,12735,8260


In [217]:
df_rfbd_clean.to_csv('clean_data/2018/df_rfbd.csv', index=False)